In [1]:
import cosima_cookbook as cc
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cmocean as cm
from scipy.optimize import curve_fit

from dask.distributed import Client

%matplotlib inline

In [2]:
client = Client(n_workers=4)
display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/46721/status,
Dashboard: /proxy/46721/status,Workers: 4
Total threads: 8,Total memory: 90.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36811,Workers: 4
Dashboard: /proxy/46721/status,Total threads: 8
Started: Just now,Total memory: 90.00 GiB
Comm: tcp://127.0.0.1:44407,Total threads: 2
Dashboard: /proxy/43737/status,Memory: 22.50 GiB
Nanny: tcp://127.0.0.1:39409,


In [3]:
### Generate new database

db = '/scratch/e14/mp2135/access-om2/archive/databases/1deg_jra55_ryf_experiments_abrupt_spinup500.db'
session = cc.database.create_session(db)
cc.querying.get_experiments(session, all=True)

,experiment,contact,email,created,description,notes,url,root_dir,ncfiles
0,1deg_jra55_ryf_cont,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,15984
1,1deg_jra55_ryf_sinusoid_x1_period2,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,4626
2,1deg_jra55_ryf_sinusoid_x1_period5,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,4626
3,1deg_jra55_ryf_sinusoid_x1_period20,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,4626
4,1deg_jra55_ryf_sinusoid_x1_period10,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,4626
5,1deg_jra55_ryf_sinusoid_x1_period7,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,2626
6,1deg_jra55_ryf_sinusoid_x1_period12,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,2626
7,1deg_jra55_ryf_sinusoid_x1_period16,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,2626
8,1deg_jra55_ryf_cont_nokb,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,7906
9,1deg_jra55_ryf_cont_kb4,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,7906


In [4]:
# directories = cc.querying.get_experiments(session, all=True)['root_dir'].to_list()
directories = ['/scratch/e14/mp2135/access-om2/archive/1deg_jra55_ryf_cool_x1_kbv',
              '/scratch/e14/mp2135/access-om2/archive/1deg_jra55_ryf_warm_x1_kbv']

cc.database.build_index(directories, session)
cc.querying.get_experiments(session, all=True)

Indexing experiment: 1deg_jra55_ryf_cool_x1_kbv


100%|██████████| 514/514 [01:00<00:00,  8.52it/s]


Indexing experiment: 1deg_jra55_ryf_warm_x1_kbv


100%|██████████| 514/514 [00:58<00:00,  8.83it/s]


,experiment,contact,email,created,description,notes,url,root_dir,ncfiles
0,1deg_jra55_ryf_cont,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,15984
1,1deg_jra55_ryf_sinusoid_x1_period2,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,4626
2,1deg_jra55_ryf_sinusoid_x1_period5,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,4626
3,1deg_jra55_ryf_sinusoid_x1_period20,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,4626
4,1deg_jra55_ryf_sinusoid_x1_period10,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,4626
5,1deg_jra55_ryf_sinusoid_x1_period7,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,2626
6,1deg_jra55_ryf_sinusoid_x1_period12,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,2626
7,1deg_jra55_ryf_sinusoid_x1_period16,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,2626
8,1deg_jra55_ryf_cont_nokb,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,7906
9,1deg_jra55_ryf_cont_kb4,None,None,None,None,None,None,/scratch/e14/mp2135/access-om2/archive/1deg_jr...,7906
